In [1]:
# libs necessárias
from c8y_api import CumulocityApi
from datetime import timedelta
import pandas as pd 



In [2]:
import json
import os

CREDENTIALS = os.path.join(os.getcwd(),'..','..','..','','configs','config.json') 

with open(CREDENTIALS) as config_file:
    login = json.load(config_file)

In [3]:
# instâncias gloabais
c8y = CumulocityApi(
    base_url= login['url'],
    tenant_id= login['tenant'],
    username= login['username'],
    password= login['password'],
)

#devices de teste
# 621151045972 -> unilever || 1069740447 -> serbom || 611284292311 -> garoto cag 09 || 481304022994 -> garoto cag 10
sources = ['621151045972', '1069740447', '611284292311','481304022994']

# métricas de hora para separar o periodo das horas measurements puxadas
two_hours = timedelta(hours=2) # 2 hours older
one_hour = timedelta(hours=1) # 1 hour older

In [8]:
one_hour_measu = c8y.measurements.select(source=sources[2],max_age=two_hours,min_age= one_hour,page_number=1,page_size=1)

for i,v in enumerate(one_hour_measu):
    print(v.fragments)

{'EB_CAG09': {'0': {'unit': None, 'value': 8.160805}}}


In [32]:
# puxando measurements de uma hora atrás
one_hour_measu = c8y.measurements.select(source=sources[2],max_age=two_hours,min_age= one_hour)
df_1 = {}
for i,v in enumerate(one_hour_measu):
    fragments = v.fragments # caracteristicas do dados puxado. Daí tiramos as propriedes que queremos
    col_name = list(fragments.keys())[0] # nome da nossa coluna no df
    value = [fragments[col_name][list(fragments[col_name].keys())[0]]['value']] # valor do dado puxado
    

    try:
        df_1[col_name].append(value[0])
    except KeyError as error:
        df_1[col_name] = value
    
    






In [4]:

'''
# Crie um novo dicionário com listas de comprimento mínimo
dados_comprimento_minimo = {chave: lista[:comprimento_minimo] for chave, lista in dados.items()}

'''
comp_min = min(map(len,df_1.values()))

for k,v in df_1.items():
    df_1[k] = v[:comp_min]


In [5]:
df_1 = pd.DataFrame(df_1)
df_1

,EB_CAG09,LB_CAG09,CO_CAG09,CI_CAG09,SP_CAG09,DP_CAG09,ST_CAG09,DT_CAG09,OT_CAG09,SSH_CAG09,...,HZ_CAG09,Horimetro_CAG09,S01_AC_01_CAG09,S02_AC_01_CAG09,S03_AC_01_CAG09,S04_AC_01_CAG09,S01_VEL_01_CAG09,S02_VEL_01_CAG09,S03_VEL_01_CAG09,S04_VEL_01_CAG09
0,8.468983,4.062244,32.41653,28.25712,3.702929,13.01373,4.301393,73.44390,51.82632,2.512353,...,51.32542,38068.0,1033.3040,1064.135,422.2708,316.4073,3.556486,1.850771,0.840039,0.748808
1,8.150990,3.915024,32.33802,28.12756,3.669245,12.94801,4.185578,73.50866,51.88815,2.595778,...,51.35300,38068.0,1025.5780,1092.576,400.2816,335.7680,3.567368,1.860974,0.728471,0.719692
2,8.470944,4.034760,32.26343,28.02157,3.708111,12.94923,4.160058,73.44979,51.79099,2.341571,...,51.17469,38068.0,1015.3340,1086.917,397.9703,322.0981,3.789294,1.912468,0.591453,0.568897
3,8.255024,3.916985,32.20847,27.99409,3.680904,12.92550,4.128652,73.40266,51.82043,2.467130,...,51.17559,38068.0,1029.4970,1099.275,426.8381,319.4786,3.738302,1.906664,0.646019,0.574145
4,7.511078,3.296707,32.15742,28.05886,3.623685,12.85917,3.891141,73.36439,51.83221,2.570680,...,50.37320,38068.0,1040.3240,1080.349,364.1911,293.7280,3.609444,1.828450,0.652584,0.526260
5,8.017510,3.638252,32.11424,28.04513,3.686950,12.85491,4.022656,73.12000,51.70560,2.328678,...,49.76666,38068.0,1037.3230,1089.060,408.4740,317.3208,4.185211,2.101417,0.680251,0.634956
6,7.938995,3.732475,32.01021,28.00979,3.686519,12.84335,4.063882,72.83735,51.53188,2.372443,...,49.13741,38068.0,1112.0840,1065.718,391.1024,311.2396,5.387774,2.554899,0.716919,0.762216
7,7.783924,3.473370,31.90028,27.96464,3.694508,12.72834,3.999100,72.55469,51.43177,2.260655,...,47.99819,38068.0,1152.3100,1119.875,366.5733,288.2392,8.769262,3.876338,1.777719,1.703809
8,7.734856,3.626472,31.87476,27.94109,3.708111,12.73321,4.099210,72.22786,51.24333,2.280723,...,47.32497,38068.0,1122.7700,1160.918,411.3620,310.6100,8.616764,3.657714,2.212162,2.086502
9,7.885998,3.759953,31.81588,27.92146,3.745465,12.61394,4.175767,71.90692,50.98717,2.137489,...,46.61749,38068.0,1061.4370,1197.644,432.7909,294.1667,5.934290,2.715564,1.940745,1.705697
